# RQ2: In which phase of the technology life cycle path are the technologies?
- Interest in topic: #Papers on a topic over time 
- Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
- Publikationstyp: Workshop -> Conference -> Journal 

## Bibliotheken einbinden

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Daten einlesen

In [2]:
data = pd.read_csv('./data/data.csv',usecols = ['Publication Year','Article Citation Count','Author Keywords','Publisher'], dtype={'Author Keywords': str, 'Publisher': str})
data.head()

,Publication Year,Author Keywords,Article Citation Count,Publisher
0,2016,software engineering education;athletic softwa...,5.0,IEEE
1,2009,software engineering;knowledge management;lear...,28.0,IEEE
2,2016,global software engineering;agile software dev...,11.0,IEEE
3,2011,software engineering;computer and information ...,11.0,IEEE
4,2017,automotive software engineering;ASE;AUTOSAR;st...,4.0,IEEE


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2165 entries, 0 to 2164
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Publication Year        2165 non-null   int64  
 1   Author Keywords         1976 non-null   object 
 2   Article Citation Count  1578 non-null   float64
 3   Publisher               2165 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 67.8+ KB


In [4]:
def IntrestInTopic(year):
    databyyear = data[data['Publication Year'] == year]
    

In [5]:
def FilterListErstellen(l):
    wordlist = []
    for x in l:
        wordlist.append(x[0])
    return pd.Series(wordlist,dtype=str)
    

In [6]:

citation_matrix_topics = data.pivot_table(index='Author Keywords', 
                        columns='Publication Year', 
                        values='Article Citation Count',
                        aggfunc=np.sum,
                        fill_value = 0)
    
citation_matrix_topics.head()
#citation_matrix_topics.info()

Publication Year,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Author Keywords,,,,,,,,,,,,,
- D.2.1Requirements/Specifications D.2.9.d Initiation and scope definition D.2.9 Management;Requirements/specifications;initiation and scope definition;management,0,0,0,0,0,0,0,12,0,0,0,0,0
11th International Symposium on Empirical Software Engineering and Measurement;13th International Conference on Predictive Models and Data Analytics in Software Engineering;21st International Systems and Software Product Line Conference;empirical software engineering;predictive models;data analytics;software product lines;product line engineering;developer productivity;issue resolution;agile development;microservices;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
33rd International Conference on Software Maintenance and Evolution;ICSME 17;32nd International Conference on Automated Software Engineering;ASE 17;software maintenance;software evolution;automated software engineering;flaky tests;technical debt;self-admitted technical debt;SATD;QA bots;regular expressions;regexes;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
40th International Conference on Software Engineering;17th International Conference on Software Reuse;gender;gender stereotypes;GenderMag;sentiment;sentiment analysis;Stack Overflow;Senti4SD;software product lines;SPL;software product line engineering;SPLE;variability management;safety-critical systems;software engineering education;software reuse;software development;software engineering;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,1,0,0,0
API Documentation;Code Sample;Usage Scenario;Stack Overflow;Crowd knowledge,0,0,0,0,0,0,0,0,0,0,3,0,0


In [7]:
stopwords = ['software engineering','software development','Software Engineering','Software engineering']

In [8]:
data["count"] = 1
matrix_topic_count = data.pivot_table(index='Author Keywords', 
                        columns='Publication Year', 
                        values='count',
                        aggfunc=np.sum,
                        fill_value = 0)

In [9]:
def KeywordTrennen(matrix,cnt,citation):
    words = {}
    data = pd.DataFrame()
    i = 0

    for year in range(2009,2022,1):
        for keywords,count in matrix[year].items():
            i = i + 1
            keywordlist = keywords.split(';')
            if count > 0:
                for keyword in keywordlist:
                    if not keyword in stopwords:
                        #if keyword == 'software engineering':
                        #    print(f"{keywords}\n {count}\n {keyword}")
                        if citation:
                            cnt[keyword] += count
                        else:
                            cnt[keyword] += 1

                        keys = list(words.keys())
                        if keyword not in keys:
                            words[keyword] = 0
                        words[keyword] += count

        seriesyear = pd.Series(words).to_frame()
        seriesyear.columns = [year]
        data = data.merge(seriesyear, how= 'outer' , left_index = True, right_index = True)
        words = {}   
    return data   

In [10]:
cnt_citation = Counter()

In [11]:
keyword_citation = KeywordTrennen(citation_matrix_topics,cnt_citation,True)  

In [12]:
keyword_citation 

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
automatic programming,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
coding tools and techniques,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
distribution,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
refactoring tools,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
workflow automation,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
workflow management,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
workflow-based systems,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
working time of developers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [13]:
anzahl_anzeigen = 40

In [14]:
cnt_citation.most_common(anzahl_anzeigen)

[('machine learning', 1398),
 ('software quality', 1002),
 ('software architecture', 869),
 ('software testing', 806),
 ('Mutation testing', 802),
 ('analysis', 743),
 ('survey', 699),
 ('Modeling', 689),
 ('survey.', 684),
 ('communication', 672),
 ('empirical studies', 651),
 ('mining software repositories', 644),
 ('DevOps', 632),
 ('defect prediction', 627),
 ('empirical study', 625),
 ('visualization', 621),
 ('microservices', 619),
 ('software', 613),
 ('diagrams', 596),
 ('visual syntax', 596),
 ('concrete syntax.', 596),
 ('Test oracle', 568),
 ('Internet of Things', 567),
 ('cloud computing', 559),
 ('product metrics', 525),
 ('Empirical software engineering', 520),
 ('refactoring', 512),
 ('IoT', 508),
 ('software metrics', 505),
 ('Privacy', 498),
 ('software product lines', 497),
 ('Software testing', 466),
 ('Systematic literature review', 466),
 ('software fault prediction', 462),
 ('agile software development', 455),
 ('verification', 453),
 ('maintainability', 448),
 ('

In [15]:
filterlist = FilterListErstellen(list(cnt_citation.most_common(anzahl_anzeigen)))

In [16]:
keyword_citation_data = keyword_citation[keyword_citation.index.isin(filterlist)].fillna(0)

In [17]:
#keyword_citation_data["Sum"] = keyword_citation_data.sum(axis = 1)

In [18]:
keyword_citation_data.head(anzahl_anzeigen)

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
DevOps,0.0,0.0,0.0,0.0,0.0,0.0,97.0,499.0,6.0,27.0,3.0,0.0,0.0
Empirical software engineering,0.0,46.0,67.0,0.0,311.0,90.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0
Internet of Things,0.0,0.0,0.0,0.0,0.0,0.0,27.0,198.0,332.0,9.0,0.0,1.0,0.0
IoT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,179.0,320.0,9.0,0.0,0.0,0.0
Modeling,689.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mutation testing,0.0,0.0,684.0,0.0,0.0,80.0,25.0,0.0,0.0,13.0,0.0,0.0,0.0
Privacy,428.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Requirements/Specifications,431.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Software testing,0.0,61.0,46.0,74.0,47.0,101.0,95.0,14.0,5.0,0.0,7.0,16.0,0.0
Systematic literature review,0.0,0.0,0.0,462.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0


In [19]:
import altair as alt
import numpy as np
import pandas as pd

keyword = keyword_citation_data.values

# Compute x^2 + y^2 across a 2D grid
year, y = np.meshgrid(range(0, 13), range(0, keyword_citation_data.index.size))
z = keyword

# Convert this grid to columnar data expected by Altair
source = pd.DataFrame({'x': year.ravel(),
                     'y': y.ravel(),
                     'z': z.ravel()})

alt.Chart(source).mark_rect().encode(
    #alt.X(field='Years', type='nominal',
    #      sort=keyword_citation_data.columns),
    #alt.Y(field='Topics', type='nominal',
    #      sort=keyword_citation_data.index.values),
    x='x:O',
    y='y:O',
    color='z:Q'
).properties(
    title='Heatmap')

alt.Chart(...)

In [20]:
cnt_topic = Counter()

In [21]:
keyword_count = KeywordTrennen(matrix_topic_count,cnt_topic,False)  

In [22]:
filterlist = FilterListErstellen(list(cnt_topic.most_common(anzahl_anzeigen)))

In [23]:
keyword_count_data = keyword_count[keyword_count.index.isin(filterlist)].fillna(0)

In [24]:
keyword_count_data.head(anzahl_anzeigen)

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Android,0.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,2.0,0.0,1.0,3.0,3.0
DevOps,0.0,0.0,0.0,0.0,0.0,0.0,3.0,8.0,4.0,12.0,1.0,2.0,0.0
IEEE Software,2.0,0.0,3.0,1.0,0.0,0.0,4.0,4.0,4.0,7.0,0.0,0.0,0.0
Internet of Things,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,8.0,3.0,0.0,1.0,0.0
Software Engineering Radio,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,3.0,5.0,0.0,0.0,0.0
Software testing,0.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,0.0,2.0,6.0,2.0
agile,0.0,2.0,4.0,1.0,2.0,2.0,1.0,4.0,1.0,3.0,0.0,1.0,0.0
agile development,1.0,2.0,0.0,1.0,0.0,0.0,0.0,6.0,3.0,13.0,0.0,0.0,0.0
agile software development,4.0,1.0,1.0,0.0,1.0,1.0,1.0,4.0,2.0,6.0,1.0,0.0,0.0
architecture,0.0,4.0,5.0,1.0,0.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
import altair as alt
import numpy as np
import pandas as pd

keyword = keyword_count_data.values

# Compute x^2 + y^2 across a 2D grid
x, y = np.meshgrid(range(2009, 2022,1), keyword_citation_data.index)
z = keyword

# Convert this grid to columnar data expected by Altair
source = pd.DataFrame({'Topics': x.ravel(),
                     'Years': y.ravel(),
                     'z': z.ravel()})

years = list(keyword_citation_data.columns.astype(str))
print(type(years))

alt.Chart(source).mark_rect().encode(
    #alt.X(field='years', type='ordinal', title='Years'),
    #alt.Y(field='keyword_citation_data.index.values', type='ordinal', title='Keyword'),
    x='Topics:O',
    y='Years:O',
    color='z:Q'
).properties(
    title='Heatmap'
)

<class 'list'>


alt.Chart(...)

In [26]:
cnt_topic.most_common(anzahl_anzeigen)

[('software architecture', 81),
 ('software', 65),
 ('software testing', 49),
 ('requirements engineering', 41),
 ('software quality', 40),
 ('DevOps', 30),
 ('empirical software engineering', 28),
 ('empirical study', 28),
 ('software maintenance', 27),
 ('machine learning', 27),
 ('refactoring', 27),
 ('requirements', 26),
 ('agile development', 26),
 ('IEEE Software', 24),
 ('software design', 24),
 ('software requirements', 22),
 ('agile software development', 22),
 ('technical debt', 22),
 ('static analysis', 22),
 ('Software testing', 21),
 ('software product lines', 20),
 ('reliability', 20),
 ('agile', 20),
 ('mining software repositories', 20),
 ('survey', 19),
 ('cloud computing', 19),
 ('Android', 19),
 ('model checking', 18),
 ('security', 18),
 ('Internet of Things', 18),
 ('architecture', 17),
 ('software analytics', 17),
 ('software reliability', 16),
 ('testing', 16),
 ('open source', 16),
 ('defect prediction', 16),
 ('Software Engineering Radio', 16),
 ('debugging', 1

In [27]:
# Top 10 Themen pro Jahr

In [28]:
#import spacy
#from spacy.symbols import NOUN


#nlp = spacy.load("en_core_web_sm")

#cnt = Counter()
#tokenlist = []

#for columns in data.itertuples():

#    text = columns[2] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
#    if type(text) is str:
#        doc = nlp(text)
        
#        for token in doc:
            # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
#            token_text = token.text
#            token_pos = token.pos_
#            token_dep = token.dep_
#            token_ent = token.ent_type_
            
#            if token.pos == NOUN:
#                if not token.is_stop:
#                    #print(f"{token_text} | {token_ent} | {token.tag_}")
#                    cnt[token_text] += 1
#                    tokenlist.append([token_text,columns[2],token_ent])
